# Basic usage

`SciKeras` is designed to maximize interoperability between `sklearn` and `Keras/TensorFlow`. The aim is to keep 99% of the flexibility of `Keras` while being able to leverage most features of `sklearn`. Below, we show the basic usage of `SciKeras` and how it can be combined with `sklearn`.

This notebook shows you how to use the basic functionality of `SciKeras`.

## Table of contents

* [1. Setup](#1.-Setup)
* [2. Training a classifier and making predictions](#2.-Training-a-classifier-and-making-predictions)
  * [2.1 A toy binary classification task](#2.1-A-toy-binary-classification-task)
  * [2.2 Definition of the Keras classification Model](#2.2-Definition-of-the-Keras-classification-Model)
  * [2.3 Defining and training the neural net classifier](#2.3-Defining-and-training-the-neural-net-classifier)
  * [2.4 Making predictions, classification](#2.4-Making-predictions-classification)
* [3 Training a regressor](#3.-Training-a-regressor)
  * [3.1 A toy regression task](#3.1-A-toy-regression-task)
  * [3.2 Definition of the Keras regression Model](#3.2-Definition-of-the-Keras-regression-Model)
  * [3.3 Defining and training the neural net regressor](#3.3-Defining-and-training-the-neural-net-regressor)
  * [3.4 Making predictions, regression](#3.4-Making-predictions-regression)
* [4. Saving and loading a model](#4.-Saving-and-loading-a-model)
  * [4.1 Saving the whole model](#4.1-Saving-the-whole-model)
  * [4.2 Saving using Keras' saving methods](#4.2-Saving-using-Keras-saving-methods)
* [5. Usage with an sklearn Pipeline](#5.-Usage-with-an-sklearn-Pipeline)
* [6. Callbacks](#6.-Callbacks)
* [7. Usage with sklearn GridSearchCV](#7.-Usage-with-sklearn-GridSearchCV)
  * [7.1 Special prefixes](#7.1-Special-prefixes)
  * [7.2 Performing a grid search](#7.2-Performing-a-grid-search)

## 1. Setup

In [1]:
try:
    import scikeras
except ImportError:
    !python -m pip install scikeras

Silence TensorFlow logging to keep output succinct.

In [2]:
import warnings
from tensorflow import get_logger
get_logger().setLevel('ERROR')
warnings.filterwarnings("ignore", message="Setting the random state for TF")

In [3]:
import numpy as np
from scikeras.wrappers import KerasClassifier, KerasRegressor
from tensorflow import keras

## 2. Training a classifier and making predictions

### 2.1 A toy binary classification task

We load a toy classification task from `sklearn`.

In [4]:
import numpy as np
from sklearn.datasets import make_classification

In [5]:
X, y = make_classification(1000, 20, n_informative=10, random_state=0)

In [6]:
X.shape, y.shape, y.mean()

((1000, 20), (1000,), 0.5)

### 2.2 Definition of the Keras classification Model

We define a vanilla neural network with.

Because we are dealing with 2 classes, the output layer can be constructed in
two different ways:

1. Single unit with a `"sigmoid"` nonlinearity. The loss must be `"binary_crossentropy"`.
2. Two units (one for each class) and a `"softmax"` nonlinearity. The loss must be `"sparse_categorical_crossentropy"`.

In this example, we choose the first option, which is what you would usually
do for binary classification. The second option is usually reserved for when
you have >2 classes.

In [7]:
from tensorflow import keras

In [8]:
def get_clf(meta, hidden_layer_sizes, dropout):
    n_features_in_ = meta["n_features_in_"]
    n_classes_ = meta["n_classes_"]
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(n_features_in_,)))
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    return model

### 2.3 Defining and training the neural net classifier

We use `KerasClassifier` because we're dealing with a classifcation task. The first argument should be a callable returning a `Keras.Model`, in this case, `get_clf`. As additional arguments, we pass the number of loss function (required) and the optimizer, but the later is optional. We must also pass all of the arguments to `get_clf` as keyword arguments to `KerasClassifier` if they don't have a default value in `get_clf`. Note that if you do not pass an argument to `KerasClassifier`, it will not be avilable for hyperparameter tuning. Finally, we also pass `random_state=0` for reproducible results.

In [9]:
from scikeras.wrappers import KerasClassifier

In [10]:
clf = KerasClassifier(
    model=get_clf,
    loss="binary_crossentropy",
    optimizer="adam",
    hidden_layer_sizes=(100,),
    dropout=0.5,
)

As in `sklearn`, we call `fit` passing the input data `X` and the targets `y`.

In [11]:
clf.fit(X, y)

 1/32 [..............................] - ETA: 12s - loss: 0.9246

32/32 [==============================] - 0s 1ms/step - loss: 0.8663


KerasClassifier(
	model=<function get_clf at 0x7f491fe05c10>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=binary_crossentropy
	metrics=None
	batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	hidden_layer_sizes=(100,)
	dropout=0.5
	class_weight=None
)

Also, as in `sklearn`, you may call `predict` or `predict_proba` on the fitted model.

### 2.4 Making predictions, classification

In [12]:
y_pred = clf.predict(X[:5])
y_pred

1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 63ms/step


array([0, 0, 0, 0, 0])

In [13]:
y_proba = clf.predict_proba(X[:5])
y_proba

1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 12ms/step


array([[0.59548366, 0.40451634],
       [0.55340177, 0.44659823],
       [0.933987  , 0.06601298],
       [0.891415  , 0.10858503],
       [0.8329    , 0.16710001]], dtype=float32)

## 3 Training a regressor

### 3.1 A toy regression task

In [14]:
from sklearn.datasets import make_regression

In [15]:
X_regr, y_regr = make_regression(1000, 20, n_informative=10, random_state=0)

In [16]:
X_regr.shape, y_regr.shape, y_regr.min(), y_regr.max()

((1000, 20), (1000,), -649.0148244404172, 615.4505181286091)

### 3.2 Definition of the Keras regression Model

Again, define a vanilla neural network. The main difference is that the output layer always has a single unit and does not apply any nonlinearity.

In [17]:
def get_reg(meta, hidden_layer_sizes, dropout):
    n_features_in_ = meta["n_features_in_"]
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(n_features_in_,)))
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1))
    return model

### 3.3 Defining and training the neural net regressor

Training a regressor is almost the same as training a classifier. Mainly, we use `KerasRegressor` instead of `KerasClassifier` (this is the same terminology as in `sklearn`). We also change the loss function to `KerasRegressor.r_squared`. SciKeras provides this loss function because most of the `sklearn` ecosystem expects `R^2` as the loss function, but Keras does not have a default implementation.

In [18]:
from scikeras.wrappers import KerasRegressor

In [19]:
reg = KerasRegressor(
    model=get_reg,
    loss=KerasRegressor.r_squared,
    optimizer="adam",
    hidden_layer_sizes=(100,),
    dropout=0.5,
)

In [20]:
reg.fit(X_regr, y_regr)

 1/32 [..............................] - ETA: 12s - loss: -8.2731e-04

32/32 [==============================] - 0s 1ms/step - loss: -0.0155 


KerasRegressor(
	model=<function get_reg at 0x7f48fdfed820>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=<function KerasRegressor.r_squared at 0x7f491fe05b80>
	metrics=None
	batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	hidden_layer_sizes=(100,)
	dropout=0.5
)

### 3.4 Making predictions, regression

You may call `predict` or `predict_proba` on the fitted model. For regressions, both methods return the same value.

In [21]:
y_pred = reg.predict(X_regr[:5])
y_pred

1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 38ms/step


array([-0.0653416 ,  0.41834956,  0.90648997,  0.15211703, -0.40553126],
      dtype=float32)

## 4. Saving and loading a model

Save and load either the whole model by using pickle, or use Keras' specialized save methods on the `KerasClassifier.model_` or `KerasRegressor.model_` attribute that is created after fitting. You will want to use Keras' model saving utilities if any of the following apply:

1. You wish to save only the weights or only the training configuration of your model.
2. You wish to share your model with collaborators. Pickle is a relatively unsafe protocol and it is not recommended to share or load pickle objects publically.
3. You care about performance, especially if doing in-memory serialization.

For more information, see Keras' [saving documentation](https://www.tensorflow.org/guide/keras/save_and_serialize).

### 4.1 Saving the whole model

In [22]:
import pickle

In [23]:
bytes_model = pickle.dumps(reg)

In [24]:
new_reg = pickle.loads(bytes_model)
new_reg

KerasRegressor(
	model=<function get_reg at 0x7f48fdfed820>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=<function KerasRegressor.r_squared at 0x7f491fe05b80>
	metrics=None
	batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	hidden_layer_sizes=(100,)
	dropout=0.5
)

### 4.2 Saving using Keras' saving methods

This efficiently and safely saves the model to disk, including trained weights.
You should use this method if you plan on sharing your saved models.

In [25]:
# Save to disk
reg.model_.save("/tmp/my_model")

In [26]:
# Load the model back into memory
new_reg_model = keras.models.load_model("/tmp/my_model")
# Now we need to instantiate a new SciKeras object with this model
# Note that we no longer pass paramters like hidden_layer_sizes, those
# are note "fixed"
reg = KerasRegressor(
    new_reg_model,
    loss=KerasRegressor.r_squared,
    optimizer="adam",
)
reg.fit(X_regr, y_regr)
reg.predict(X_regr[:5])

 1/32 [..............................] - ETA: 9s - loss: -0.0207

32/32 [==============================] - 0s 1ms/step - loss: -0.0460


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 30ms/step


array([-0.40252566,  0.66959363,  0.86581403, -0.04017418, -0.5535241 ],
      dtype=float32)

## 5. Usage with an sklearn Pipeline

It is possible to put the `KerasClassifier` inside an `sklearn Pipeline`, as you would with any `sklearn` classifier.

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [28]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('clf', clf),
])

In [29]:
pipe.fit(X, y)

 1/32 [..............................] - ETA: 10s - loss: 0.8225

32/32 [==============================] - 0s 1ms/step - loss: 0.7456


Pipeline(steps=[('scale', StandardScaler()),
                ('clf',
                 KerasClassifier(dropout=0.5, hidden_layer_sizes=(100,), loss='binary_crossentropy', model=<function get_clf at 0x7f491fe05c10>, optimizer='adam'))])

In [30]:
y_proba = pipe.predict_proba(X[:5])
y_proba

1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 42ms/step


array([[0.28717816, 0.71282184],
       [0.6697245 , 0.33027548],
       [0.6465628 , 0.35343722],
       [0.44401318, 0.5559868 ],
       [0.63679266, 0.36320734]], dtype=float32)

To save the whole pipeline, including the Keras model, use `pickle`.

## 6. Callbacks

Adding a new callback to the model is straightforward. Below we show how to add an `EarlyStopping` callback to prevent overfitting.

In [31]:
es = keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', patience=200, verbose=1)

We now generate a toy dataset using `sklearn.datasets.make_moons`. This dataset was chosen specifically to trigger early stopping.

In [32]:
from sklearn.datasets import make_moons

In [33]:
X, y = make_moons(n_samples=100, noise=0.2, random_state=0)
X.shape, y.shape

((100, 2), (100,))

We will first check fitting without the callback and then with. We will compare the training time and final accuracy.

In [34]:
import time

In [35]:
# First test without the callback
clf = KerasClassifier(
    model=get_clf,
    loss="binary_crossentropy",
    optimizer="adam",
    hidden_layer_sizes=(500,),
    dropout=0.5,
    metrics=["binary_accuracy"],
    fit__validation_split=0.2,
    epochs=500,
    verbose=False,
)
start = time.time()
clf.fit(X, y)
print(f"Training time: {time.time() * start}")
print(f"Final accuracy: {clf.history_['val_binary_accuracy'][-1]}")  # get last value of last fit/partial_fit call

Training time: 2.5987947115066127e+18
Final accuracy: 1.0


In [36]:
# Test with the callback
clf = KerasClassifier(
    model=get_clf,
    loss="binary_crossentropy",
    optimizer="adam",
    hidden_layer_sizes=(500,),
    dropout=0.5,
    metrics=["binary_accuracy"],
    fit__validation_split=0.2,
    epochs=500,
    verbose=False,
    callbacks=[es]
)
start = time.time()
clf.fit(X, y)
print(f"Training time: {time.time() * start}")
print(f"Final accuracy: {clf.history_['val_binary_accuracy'][-1]}")  # get last value of last fit/partial_fit call

Epoch 00217: early stopping
Training time: 2.598794756054604e+18
Final accuracy: 0.949999988079071


For information on how to write custom callbacks, have a look at the 

---

[Advanced_Usage](https://nbviewer.jupyter.org/github/adriangb/scikeras/blob/master/notebooks/Advanced_Usage.ipynb) notebook.

## 7. Usage with sklearn GridSearchCV

### 7.1 Special prefixes

SciKeras allows to direct access to all parameters passed to the wrapper constructors, including deeply nested routed parameters. This allows tunning of
paramters like `hidden_layer_sizes` as well as `optimizer__learning_rate`.

This is exactly the same logic that allows to access estimator parameters in `sklearn Pipeline`s and `FeatureUnion`s.

This feature is useful in several ways. For one, it allows to set those parameters in the model definition. Furthermore, it allows you to set parameters in an `sklearn GridSearchCV` as shown below.

To differentiate paramters like `callbacks` which are accepted by both `tf.keras.Model.fit` and `tf.keras.Model.predict` you can add a `fit__` or `predict__` routing suffix respectively. Similar, the `model__` prefix may be used to specify that a paramter is destined only for `get_clf`/`get_reg` (or whatever callable you pass as your `model` argument).

For more information on parameter routing with special prefixes, see the [Advanced Usage Docs](https://scikeras.org.readthedocs.build/en/latest/advanced.html#routed-parameters)

### 7.2 Performing a grid search

Below we show how to perform a grid search over the learning rate (`optimizer__lr`), the model's number of hidden layers (`model__hidden_layer_sizes`), the model's dropout rate (`model__dropout`).

In [37]:
from sklearn.model_selection import GridSearchCV

In [38]:
clf = KerasClassifier(
    model=get_clf,
    loss="binary_crossentropy",
    optimizer="adam",
    optimizer__lr=0.1,
    model__hidden_layer_sizes=(100,),
    model__dropout=0.5,
    verbose=False,
)

*Note*: We set the verbosity level to zero (`verbose=False`) to prevent too much print output from being shown.

In [39]:
params = {
    'optimizer__lr': [0.05, 0.1],
    'model__hidden_layer_sizes': [(100, ), (50, 50, ), (33, 33, 33, )],
    'model__dropout': [0, 0.5],
}

In [40]:
gs = GridSearchCV(clf, params, scoring='accuracy', n_jobs=-1, verbose=True)

In [41]:
gs.fit(X, y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=KerasClassifier(loss='binary_crossentropy', model=<function get_clf at 0x7f491fe05c10>, model__dropout=0.5, model__hidden_layer_sizes=(100,), optimizer='adam', optimizer__lr=0.1, verbose=False),
             n_jobs=-1,
             param_grid={'model__dropout': [0, 0.5],
                         'model__hidden_layer_sizes': [(100,), (50, 50),
                                                       (33, 33, 33)],
                         'optimizer__lr': [0.05, 0.1]},
             scoring='accuracy', verbose=True)

In [42]:
print(gs.best_score_, gs.best_params_)

0.8699999999999999 {'model__dropout': 0, 'model__hidden_layer_sizes': (100,), 'optimizer__lr': 0.1}


Of course, we could further nest the `KerasClassifier` within an `sklearn Pipeline`, in which case we just prefix the parameter by the name of the net (e.g. `clf__model__hidden_layer_sizes`).